In [1]:
#Importation des librairies necessaires
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#Extraire les features (mfcc, chroma, mel) a partir du sonore 
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [3]:
#Définir un dictionnaire contenant la liste des emotions disponibles dans notre dataset
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'Naturel',
  '02':'Calme',
  '03':'Heureux',
  '04':'Enervé',
  '05':'Faché',
  '06':'Craintif',
  '07':'Dégoutté',
  '08':'Surpris'
}

#Emotions A observer 
observed_emotions=['calme', 'heureux', 'Enervé', 'Dégoutté']

In [4]:
#Charger les données et extraire les features de chaque extrait sonore
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Reconnaissance-emotion\\dataset\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
#Diviser nos données en test set et train set 
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [6]:
#voir la forme de train et de testset 
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [7]:
#voir le nombre des features extraites 
print(f'Les features extraits: {x_train.shape[1]}')

Features extracted: 180


In [8]:
#Initialiser le perceptron Multi couches
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=500)

In [9]:
#Adaptatiomn du modèle
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(500,),
              learning_rate='adaptive', max_iter=500)

In [10]:
#Prediction sur le train test
y_pred=model.predict(x_test)

In [11]:
#Calculer l'accuracy de notre modèle
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Afficher l'accuracy 
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.10%
